<a href="https://colab.research.google.com/github/wathsan11/Rextro-NILM-Project/blob/main/NILM_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RANDOM_STATE = 42

In [20]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/NILM.csv')
df.head()

,voltage,current,active_power,reactive_power,apparent_power,power_factor,timestamp,reliable,device_id,load_type,probe_id,frequency,energy,temperature,bulb,fan,iron,labels
0,233.67,0.427060,84.82254,52.56829,99.79122,0.85,2025-12-09T19:02:50.725756,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
1,233.67,0.435269,86.45290,53.57870,101.70930,0.85,2025-12-09T19:02:53.781784,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
2,233.67,0.451382,89.65333,55.56215,105.47450,0.85,2025-12-09T19:02:56.810625,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
3,233.67,0.457015,90.77206,56.25548,106.79070,0.85,2025-12-09T19:02:59.867042,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1
4,233.67,0.445174,88.42026,54.79794,104.02380,0.85,2025-12-09T19:03:02.925509,False,power_monitor_1,unknown,default,50,0,25,1,0,0,1


In [4]:
x = df.drop(['timestamp','reliable','device_id','load_type','probe_id','frequency','energy','temperature','bulb','fan','iron','labels'],axis = 1)
y = df.labels

In [5]:
x.head()

,voltage,current,active_power,reactive_power,apparent_power,power_factor
0,233.67,0.427060,84.82254,52.56829,99.79122,0.85
1,233.67,0.435269,86.45290,53.57870,101.70930,0.85
2,233.67,0.451382,89.65333,55.56215,105.47450,0.85
3,233.67,0.457015,90.77206,56.25548,106.79070,0.85
4,233.67,0.445174,88.42026,54.79794,104.02380,0.85


In [7]:
y.head()

,labels
0,1
1,1
2,1
3,1
4,1


In [9]:
x.isnull().sum()

,0
voltage,0
current,0
active_power,0
reactive_power,0
apparent_power,0
power_factor,0


In [6]:
from sklearn.model_selection import train_test_split

x_train,x_,y_train,y_ = train_test_split(x,y,test_size = 0.4,random_state = RANDOM_STATE)
x_cv,x_test,y_cv,y_test = train_test_split(x_,y_,test_size = 0.5,random_state = RANDOM_STATE)
del x_,y_

In [10]:
#XGBoost

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [49]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_model = XGBClassifier(n_estimators = 500,learning_rate=0.2,verbosity=1,random_state = RANDOM_STATE)

xgb_model.fit(
    x_train,
    y_train,
    eval_set=[(x_cv,y_cv)]
)

[0]	validation_0-mlogloss:1.67242
[1]	validation_0-mlogloss:1.50158
[2]	validation_0-mlogloss:1.37744
[3]	validation_0-mlogloss:1.28422
[4]	validation_0-mlogloss:1.21120
[5]	validation_0-mlogloss:1.15198
[6]	validation_0-mlogloss:1.10429
[7]	validation_0-mlogloss:1.06587
[8]	validation_0-mlogloss:1.03474
[9]	validation_0-mlogloss:1.00783
[10]	validation_0-mlogloss:0.98527
[11]	validation_0-mlogloss:0.96721
[12]	validation_0-mlogloss:0.95089
[13]	validation_0-mlogloss:0.93705
[14]	validation_0-mlogloss:0.92627
[15]	validation_0-mlogloss:0.91637
[16]	validation_0-mlogloss:0.90823
[17]	validation_0-mlogloss:0.90100
[18]	validation_0-mlogloss:0.89578
[19]	validation_0-mlogloss:0.89190
[20]	validation_0-mlogloss:0.88633
[21]	validation_0-mlogloss:0.88264
[22]	validation_0-mlogloss:0.87994
[23]	validation_0-mlogloss:0.87828
[24]	validation_0-mlogloss:0.87628
[25]	validation_0-mlogloss:0.87451
[26]	validation_0-mlogloss:0.87254
[27]	validation_0-mlogloss:0.87188
[28]	validation_0-mlogloss:0.8

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [50]:
print(f"""Metrics train:
	Accuracy score: {accuracy_score(xgb_model.predict(x_train),y_train):.4f}
Metrics cv:
	Accuracy score: {accuracy_score(xgb_model.predict(x_cv),y_cv):.4f}
Metrics test:
	Accuracy score: {accuracy_score(xgb_model.predict(x_test),y_test):.4f}""")

Metrics train:
	Accuracy score: 0.7016
Metrics cv:
	Accuracy score: 0.5437
Metrics test:
	Accuracy score: 0.2453
